In [2]:
from dotenv import load_dotenv
load_dotenv()

import os

from agential.core.llm import LLM

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

In [ ]:
from agential.agents.reflexion.prompts import GSM8K_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT, GSM8K_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT, REFLEXION_COT_INSTRUCTION_GSM8K, REFLEXION_COT_REFLECT_INSTRUCTION_GSM8K, REFLEXION_REACT_INSTRUCTION_GSM8K, REFLEXION_REACT_REFLECT_INSTRUCTION_GSM8K
from agential.agents.reflexion.strategies.math import ReflexionCoTMathStrategy, ReflexionReActMathStrategy
from agential.agents.self_refine.prompts import GSM8K_CRITIQUE_FEWSHOT_EXAMPLES, GSM8K_REFINE_FEWSHOT_EXAMPLES, HUMANEVAL_CRITIQUE_FEWSHOT_EXAMPLES, HUMANEVAL_REFINE_FEWSHOT_EXAMPLES, SELF_REFINE_CRITIQUE_INSTRUCTION_GSM8K, SELF_REFINE_CRITIQUE_INSTRUCTION_HUMANEVAL, SELF_REFINE_INSTRUCTION_GSM8K, SELF_REFINE_INSTRUCTION_HUMANEVAL, SELF_REFINE_REFINE_INSTRUCTION_GSM8K, SELF_REFINE_REFINE_INSTRUCTION_HUMANEVAL
from agential.agents.self_refine.strategies.code import SelfRefineCodeStrategy
from agential.agents.self_refine.strategies.math import SelfRefineMathStrategy
from agential.core.fewshots.gsm8k import GSM8K_FEWSHOT_EXAMPLES_COT, GSM8K_FEWSHOT_EXAMPLES_POT, GSM8K_FEWSHOT_EXAMPLES_REACT
from agential.core.fewshots.humaneval import HUMANEVAL_FEWSHOT_EXAMPLES_POT

question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
key = "-9867630"

strategy = ReflexionCoTMathStrategy(llm=llm, testing=True)
out = strategy.generate(
    question=question,
    key=key,
    examples=GSM8K_FEWSHOT_EXAMPLES_COT,
    reflect_examples=GSM8K_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    prompt=REFLEXION_COT_INSTRUCTION_GSM8K,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_GSM8K,
    reflect_strategy="reflexion",
    additional_keys={},
    reflect_additional_keys={},
    patience=3,
    reset=True,
)

In [ ]:
out.additional_info[0]

In [ ]:
out